## __기본옵션__

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

In [ ]:
 tf.__version__ 

'2.2.0'

In [ ]:
import os, glob
import numpy as np
import pandas as pd
from PIL import Image
import pickle
import cv2
import matplotlib.pyplot as plt
import warnings 
from time import time
warnings.filterwarnings('ignore')
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization,Add,Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.layers import LeakyReLU, ReLU, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, UpSampling2D, concatenate
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.models import Sequential

Using TensorFlow backend.


In [ ]:
dir = "/content/drive/My Drive/KPMG/classifier"
os.chdir(dir)

In [ ]:
os.getcwd()

'/content/drive/My Drive/KPMG/classifier'

## __데이터 불러오기__

In [ ]:
# pickle파일 read
with open("/content/drive/My Drive/KPMG/classifier/pickle/batang.pkl","rb") as pk:
  batang_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/dotum.pkl","rb") as pk:
  dotum_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/gothic.pkl","rb") as pk:
  gothic_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/myeongjo.pkl","rb") as pk:
  myeongjo_pk = pickle.load(pk)


In [ ]:
# 전체 이미지 array 저장해놓은 것
# image_all = batang_pk+dotum_pk+gothic_pk+myeongjo_pk
batang_pk=np.array(batang_pk)
dotum_pk=np.array(dotum_pk)
gothic_pk=np.array(gothic_pk)
myeongjo_pk=np.array(myeongjo_pk)

In [ ]:
# seed설정
np.random.seed(0)

# 인덱스 만들어 주고 순서를 랜덤으로 바꾸어 인덱스를 출력해주는 함수
def make_index_permutation(font_pk):
  result=np.random.permutation(np.arange(len(font_pk)))
  return result


In [ ]:
batang_index=make_index_permutation(batang_pk)
dotum_index=make_index_permutation(dotum_pk)
gothic_index=make_index_permutation(gothic_pk)
myeongjo_index=make_index_permutation(myeongjo_pk)

In [ ]:
import math

## font_index를 slicing으로 split 해주고 리스트에 추가해주는 함수

def data_split_merge(split_cnt,font_index):
  result=[]
  for i in range(0,split_cnt):
    result.append(font_index[round((i*(len(font_index))/split_cnt)):round((i+1)*(len(font_index))/split_cnt)])
  return result


In [ ]:
split_cnt = 8

batang_index_list=data_split_merge(split_cnt,batang_index)
dotum_index_list=data_split_merge(split_cnt,dotum_index)
gothic_index_list=data_split_merge(split_cnt,gothic_index)
myeongjo_index_list=data_split_merge(split_cnt,myeongjo_index)


In [ ]:
# 원하는 갯수로 train data로 나눠주는 함수

def make_train_data(split_cnt,font_pk,font_index_list):
  result=[]
  for i in range(split_cnt):
    result.append(font_pk[font_index_list[i]])
  return result

In [ ]:
batang_train=make_train_data(split_cnt,batang_pk,batang_index_list)
dotum_train=make_train_data(split_cnt,dotum_pk,dotum_index_list)
gothic_train=make_train_data(split_cnt,gothic_pk,gothic_index_list)
myeongjo_train=make_train_data(split_cnt,myeongjo_pk,myeongjo_index_list)

total_train=[]
for i in range(split_cnt):
  total_train.append(batang_train[i].tolist()+dotum_train[i].tolist()+gothic_train[i].tolist()+myeongjo_train[i].tolist())

In [ ]:
total_df_list=[]

for i in range(split_cnt):
  batang_df= pd.DataFrame({'image':batang_train[i], 'category':0})
  dotum_df = pd.DataFrame({'image':dotum_train[i], 'category':1})
  gothic_df = pd.DataFrame({'image':gothic_train[i], 'category':2})
  myeongjo_df = pd.DataFrame({'image':myeongjo_train[i], 'category':3})
  total_df=pd.concat([batang_df,dotum_df,gothic_df,myeongjo_df])
  total_df.reset_index(drop=True, inplace=True)
  total_df_list.append(total_df)

In [ ]:
X_list=[]

def make_X_list(total_train_data):
  img=[]
  for i in range(len(total_train_data)):
    img.append(cv2.resize(total_train_data[i],(64,64)))
  return img


In [ ]:
for i in range(split_cnt):
  X_list.append(make_X_list(total_train[i]))

## __모델링 (EfficientNetB7)__

### Train, Validation Split

In [ ]:

def make_train_XY(X,total_df_list,idx):
  return X_list[idx],np.array(total_df_list[idx]['category'])

#### 아래 train_XY안에 있는 idx를 0~split_cnt-1까지 바꿔주면서 넣어주기만 하면 됨

X,Y=make_train_XY(X_list,total_df_list,0)
X=np.array(X)

In [ ]:
X.shape

(13806, 64, 64, 3)

In [ ]:
Y.shape

(13806,)

In [ ]:
# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

In [ ]:
def Time(model, train_X,train_Y, batch_size, n_epoch):

  training_start_time = time()
  trained_model = model.fit(train_X, train_Y,
                            batch_size=batch_size, epochs=n_epoch,
                            validation_split = 0.2)
  training_end_time = time()
  print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
                                                          training_end_time - training_start_time))
  return trained_model

### EfficientNetB5 모델

In [ ]:
# pip install efficientnet

In [ ]:
# EfficientNetB5 
import efficientnet.keras as efn

eff = efn.EfficientNetB5(include_top=False, input_shape=(64,64,3), classes=4)
eff.trainable=True
# eff.summary()

from keras.models import Sequential
from keras import layers

model = Sequential()
model.add(eff)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
115515392/115515256 [==============================] - 5s 0us/step


#### optimizer 변경

##### adam

In [ ]:
# EfficientNetB5
import efficientnet.keras as efn

eff = efn.EfficientNetB5(include_top=False, input_shape=(64,64,3), classes=4)
eff.trainable=True

model = Sequential()
model.add(eff)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
!pip uninstall tensorflow 2.2

In [ ]:
!pip install tensorflow==1.15

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
tf.__version__

In [ ]:
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"],options=run_opts)
fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=2)

score = model.evaluate(X_test, y_test)
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])


Train on 8835 samples, validate on 2209 samples
Epoch 1/2


In [ ]:
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

##### nadam

In [ ]:
# EfficientNetB7 
import efficientnet.keras as efn

eff = efn.EfficientNetB7(include_top=False, input_shape=(64,64,3), classes=4)
eff.trainable=True

model = Sequential()
model.add(eff)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 17670 samples, validate on 4418 samples
Epoch 1/20
17670/17670 [==============================] - 58s 3ms/step - loss: 2.6128 - accuracy: 0.3552 - val_loss: 8.1287 - val_accuracy: 0.2395
Epoch 2/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.2256 - accuracy: 0.4345 - val_loss: 15.4822 - val_accuracy: 0.2345
Epoch 3/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.1094 - accuracy: 0.4783 - val_loss: 15.4235 - val_accuracy: 0.2383
Epoch 4/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.0631 - accuracy: 0.5010 - val_loss: 10.1142 - val_accuracy: 0.2793
Epoch 5/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.0350 - accuracy: 0.5148 - val_loss: 19.5818 - val_accuracy: 0.2406
Epoch 6/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.0201 - accuracy: 0.5211 - val_loss: 12.1113 - val_accuracy: 0.2644
Epoch 7/20
17670/17670 [==============================] - 48s 3m

##### SGD

In [ ]:
# EfficientNetB7 
import efficientnet.keras as efn

eff = efn.EfficientNetB7(include_top=False, input_shape=(64,64,3), classes=4)
eff.trainable=True

model = Sequential()
model.add(eff)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 17670 samples, validate on 4418 samples
Epoch 1/20
17670/17670 [==============================] - 57s 3ms/step - loss: 1.3735 - accuracy: 0.3127 - val_loss: 10.6419 - val_accuracy: 0.3128
Epoch 2/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.3358 - accuracy: 0.3522 - val_loss: 10.0595 - val_accuracy: 0.3085
Epoch 3/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.3013 - accuracy: 0.3825 - val_loss: 12.5498 - val_accuracy: 0.3151
Epoch 4/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.2705 - accuracy: 0.4034 - val_loss: 11.9367 - val_accuracy: 0.3115
Epoch 5/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.2372 - accuracy: 0.4188 - val_loss: 14.8028 - val_accuracy: 0.3105
Epoch 6/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.2017 - accuracy: 0.4431 - val_loss: 12.8728 - val_accuracy: 0.3105
Epoch 7/20
17670/17670 [==============================] - 47s 3

##### Adadelta

In [ ]:
# EfficientNetB7 
import efficientnet.keras as efn

eff = efn.EfficientNetB7(include_top=False, input_shape=(64,64,3), classes=4)
eff.trainable=True

model = Sequential()
model.add(eff)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adadelta", metrics=["accuracy"])
fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 17670 samples, validate on 4418 samples
Epoch 1/20
17670/17670 [==============================] - 57s 3ms/step - loss: 1.4247 - accuracy: 0.3803 - val_loss: 13.9033 - val_accuracy: 0.3148
Epoch 2/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.1592 - accuracy: 0.4601 - val_loss: 12.4221 - val_accuracy: 0.2528
Epoch 3/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.0939 - accuracy: 0.4878 - val_loss: 14.2379 - val_accuracy: 0.2399
Epoch 4/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.0689 - accuracy: 0.5022 - val_loss: 10.9552 - val_accuracy: 0.2284
Epoch 5/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.0432 - accuracy: 0.5139 - val_loss: 10.4733 - val_accuracy: 0.2669
Epoch 6/20
17670/17670 [==============================] - 48s 3ms/step - loss: 1.0225 - accuracy: 0.5242 - val_loss: 9.4557 - val_accuracy: 0.2349
Epoch 7/20
17670/17670 [==============================] - 48s 3m

##### Rmsprop

In [ ]:
# EfficientNetB7 
import efficientnet.keras as efn

eff = efn.EfficientNetB7(include_top=False, input_shape=(64,64,3), classes=4)
eff.trainable=True

model = Sequential()
model.add(eff)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 17670 samples, validate on 4418 samples
Epoch 1/20
17670/17670 [==============================] - 57s 3ms/step - loss: 3.3504 - accuracy: 0.3531 - val_loss: 16.5914 - val_accuracy: 0.2098
Epoch 2/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.3726 - accuracy: 0.3899 - val_loss: 12.4493 - val_accuracy: 0.2503
Epoch 3/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.2950 - accuracy: 0.4136 - val_loss: 28.9308 - val_accuracy: 0.2442
Epoch 4/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.2105 - accuracy: 0.4538 - val_loss: 10.9208 - val_accuracy: 0.2123
Epoch 5/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.1404 - accuracy: 0.4650 - val_loss: 15.5965 - val_accuracy: 0.2506
Epoch 6/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.0917 - accuracy: 0.4891 - val_loss: 9.8606 - val_accuracy: 0.2777
Epoch 7/20
17670/17670 [==============================] - 47s 3m

##### adagrad

In [ ]:
# EfficientNetB7 
import efficientnet.keras as efn

eff = efn.EfficientNetB7(include_top=False, input_shape=(64,64,3), classes=4)
eff.trainable=True

model = Sequential()
model.add(eff)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adagrad", metrics=["accuracy"])
fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 17670 samples, validate on 4418 samples
Epoch 1/20
17670/17670 [==============================] - 57s 3ms/step - loss: 37.2521 - accuracy: 0.3087 - val_loss: 1.3842 - val_accuracy: 0.2302
Epoch 2/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.2678 - accuracy: 0.3868 - val_loss: 1.3949 - val_accuracy: 0.3207
Epoch 3/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.1951 - accuracy: 0.4191 - val_loss: 1.3865 - val_accuracy: 0.3210
Epoch 4/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.1417 - accuracy: 0.4377 - val_loss: 1.3860 - val_accuracy: 0.2388
Epoch 5/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.1231 - accuracy: 0.4538 - val_loss: 13.5374 - val_accuracy: 0.2465
Epoch 6/20
17670/17670 [==============================] - 47s 3ms/step - loss: 1.1115 - accuracy: 0.4675 - val_loss: 19.8397 - val_accuracy: 0.2392
Epoch 7/20
17670/17670 [==============================] - 47s 3ms/

#### Training

In [ ]:
# model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# def Time(model, train_X,train_Y, batch_size, n_epoch):

#   training_start_time = time()
#   trained_model = model.fit(train_X, train_Y,
#                             batch_size=batch_size, epochs=n_epoch,
#                             validation_split = 0.2)
#   training_end_time = time()
#   print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
#                                                           training_end_time - training_start_time))
#   return trained_model


# fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=20)